# <center> Workshop - Intro to MDAnalysis Part 2</center>


# Distance calculations in MDAnalysis

Atom coordinates are in the 
`.positions` attribute of an `AtomGroup`

The positions are returned as a NumPy array, which we can then readily manipulate.




In [6]:
# First we import MDAnalysis
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PSF, DCD

u = mda.Universe(PSF, DCD)

ag = u.select_atoms('protein')

pos = ag.positions

print(f'AtomGroup has length {len(ag)} and positions is shape {pos.shape}')

pos

AtomGroup has length 3341 and positions is shape (3341, 3)


array([[ 11.736044 ,   8.500797 , -10.445281 ],
       [ 12.365119 ,   7.839936 , -10.834842 ],
       [ 12.0919485,   9.441535 , -10.724611 ],
       ...,
       [  6.512604 ,  18.447018 ,  -7.134053 ],
       [  6.300186 ,  19.363485 ,  -7.935916 ],
       [  5.5854015,  17.589624 ,  -6.9656615]], dtype=float32)

Some built-in functions based on position data:

- `center_of_mass()`

- `center_of_geometry()`


In [4]:
print(ag.center_of_mass())

print(ag.center_of_geometry())

[-0.01094035  0.05727601 -0.12885778]
[-0.04223882  0.01418196 -0.03504874]


## The `lib.distances` module

Particle positions are given as numpy arrays, so most work can be done using numpy (and numpy derived) libraries.

MDAnalysis  `lib.distances` module comes handy when

- we have periodic boundary conditions (numpy cannot handle this)

- domain specific algorithms can be used


In [13]:
from MDAnalysis.lib import distances

distances

<module 'MDAnalysis.lib.distances' from '/Users/richard/mambaforge/envs/mda/lib/python3.11/site-packages/MDAnalysis/lib/distances.py'>

### `distance_array`

To calculate **all** pairwise distances between two arrays of coordinates.

In [29]:
ag1 = u.atoms[:10]
ag2 = u.atoms[10:30]


da = distances.distance_array(ag1.positions, 
                              ag2.positions,
                              box=u.dimensions)

print(f'Our input atomgroups had sizes {len(ag1)} and {len(ag2)}, the output had shape: {da.shape}')
print()

print(da)

Our input atomgroups had sizes 10 and 20, the output had shape: (10, 20)

[[4.18088681 4.14294196 5.04585143 5.32751757 4.7130687  6.30996415
  5.68400283 2.54812542 2.96635031 3.67629133 3.85846855 4.99648446
  4.94565935 5.93783435 6.90992514 6.38235495 5.82786529 5.33756183
  6.9460756  5.57071665]
 [4.91489955 4.85935014 5.98073929 6.23106959 5.54186499 7.18169306
  6.62319949 2.8273095  3.26287418 3.7922078  3.96758782 5.03623467
  4.97678969 5.74742629 6.73735139 6.22207718 5.38776855 4.86114181
  6.45646673 5.02042183]
 [4.17637449 4.53242997 5.03940839 4.97153416 4.23180203 5.9572419
  5.2354988  3.42427919 3.93015825 4.44948799 4.46483858 5.85226664
  5.89520245 6.77510548 7.78459265 7.10265153 6.67269431 6.26081198
  7.7989879  6.24766953]
 [4.76201846 4.45247434 5.1726278  5.66651264 5.21592931 6.70279043
  5.88478524 2.86477004 2.84568316 4.17159323 4.54119591 5.32984121
  5.07396141 6.36955445 7.25278687 6.95744115 6.26345124 5.61743675
  7.34000981 6.20978052]
 [3.0020826

The output of distance array is a matrix of the distance between each position in the first coordinate array and each position in the second coordinate array.

For example to look at the distance between the 4th and 6th atom in the two AtomGroups:

In [30]:
print(f'The distance between {ag1[3]} and {ag2[5]} is {da[3, 5]} A')

The distance between <Atom 4: HT3 of type 2 of resname MET, resid 1 and segid 4AKE> and <Atom 16: HE2 of type 3 of resname MET, resid 1 and segid 4AKE> is 6.70279043302332 A


### `self_distance_array`

For calculating distances between all combinations of coordinates.

Takes a **single array** of coordinates and calculates all pairwise distances.
This will yield  ½ n(n-1) distances.


In [34]:
sda = distances.self_distance_array(ag1.positions, box=None)

print(f'Our input AtomGroup had size {len(ag1)} and the output has shape {sda.shape}')


Our input AtomGroup had size 10 and the output has shape (45,)


### `calc_bonds`

For calculating a series of distances between pairs of coordinates.

Takes 2 arrays of coordinates **of equal length**, and returns the distances between coordinate pairs in each row.


In [35]:
coords1 = u.atoms[:10].positions
coords2 = u.atoms[10:20].positions
dist = distances.calc_bonds(coords1, 
                            coords2, 
                            box=None)

print(f'The inputs had length {len(coords1)} and {len(coords2)} and the output has shape {dist.shape}')
print()
print(f'The distance between the first coordinate in each array is: {dist[0]}')

The inputs had length 10 and 10 and the output has shape (10,)

The distance between the first coordinate in each array is: 4.180886814373797


# `calc_angles` & `calc_dihedrals`

Same as `calc_bonds`

Calculate either the angle or dihedral angle between 3 or 4 arrays of coordinates.

Takes 3 or 4 arrays of identical length coordinates.


In [38]:
import numpy as np
coords1 = u.atoms[:10].positions
coords2 = u.atoms[10:20].positions
coords3 = u.atoms[20:30].positions

angles = distances.calc_angles(
            coords1, coords2, coords3,
            box=None, result=None)

print(np.rad2deg(angles))


coords4 = u.atoms[30:40].positions

dihedrals = distances.calc_dihedrals(coords1, coords2, coords3, coords4)

[ 60.1705515   70.44413816  61.74292062  48.97993666  31.49563244
  31.70597945  15.62738421 162.62889398 125.52167306 148.32427929]


# Minimum image convention

To account for periodic boundary conditions in distance calculations,
pass the box information as `box=ag.dimensions` to any distance function.


# `capped_distance` and `self_capped_distance`

Only find distances up to a limit. It returns:
- an array of indices
- an array of distances


In [44]:
ix, dist = distances.capped_distance(reference, 
                          configuration, 
                          min_cutoff = 3,
                          max_cutoff=4,
                          box=u.dimensions)
ix[:3], dist[:3]

(array([[   0, 1214],
        [   0, 1209],
        [   0,  390]]),
 array([3.07942888, 3.81926148, 3.99841942]))

2022-07-26 11:19:45 2022-07-26 11:19:46 ## A summary of Lecture 1

Most simulation analysis will involve extracting position data from certain atoms.

- A `Universe` contains all information about a simulation system

- An `AtomGroup` contains information about a group of atoms

- We can use `Universe.select_atoms()` to create an `AtomGroup` containing specific atoms from a `Universe`

- Positions of atoms in an AtomGroup are accessed through `AtomGroup.positions`

# A summary of Lecture 2

Calculating pairwise distances:
- calc_bonds
- distance_array
- self_distance_array

Faster, sparse pairwise distances:
- capped_distance
- self_capped_distance

Calculating angles:
- calc_angles
- calc_dihedrals


## Now - on to the second tutorial!

Find the tutorial notebook `Tutorial2_Distances_Trajectories` under: https://github.com/MDAnalysis/MDAnalysisWorkshop2023/notebooks/Tutorial2_Distances_Trajectories.ipynb

**Remember:**
- Go at your own pace!
- Ask questions!
- Take breaks!